In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# preprocessing/decomposition
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA, FastICA, FactorAnalysis, KernelPCA

# keras 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor

# model evaluation
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# supportive models
from sklearn.ensemble import GradientBoostingRegressor
# feature selection (from supportive model)
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV

# to make results reproducible
seed = 42 # was 42

import os 

Using TensorFlow backend.


In [2]:
INPUT = '/media/tin/DATA/Kaggle/Mercedes/Input/'
OUTPUT = '/media/tin/DATA/Kaggle/Mercedes/Output/'

# Read datasets
train = pd.read_csv(os.path.join(INPUT, 'train.csv'))
test = pd.read_csv(os.path.join(INPUT, 'test.csv'))

# save IDs for submission
id_test = test['ID'].copy()

# glue datasets together
total = pd.concat([train, test], axis=0)
print('initial shape: {}'.format(total.shape))

# binary indexes for train/test set split
is_train = ~total.y.isnull()

# find all categorical features
cf = total.select_dtypes(include=['object']).columns

# make one-hot-encoding convenient way - pandas.get_dummies(df) function
dummies = pd.get_dummies(
    total[cf],
    drop_first=False # you can set it = True to ommit multicollinearity (crucial for linear models)
)

print('oh-encoded shape: {}'.format(dummies.shape))

# get rid of old columns and append them encoded
total = pd.concat(
    [
        total.drop(cf, axis=1), # drop old
        dummies # append them one-hot-encoded
    ],
    axis=1 # column-wise
)

print('appended-encoded shape: {}'.format(total.shape))

# recreate train/test again, now with dropped ID column
train, test = total[is_train].drop(['ID'], axis=1), total[~is_train].drop(['ID', 'y'], axis=1)

# drop redundant objects
del total

# check shape
print('\nTrain shape: {}\nTest shape: {}'.format(train.shape, test.shape))

initial shape: (8418, 378)
oh-encoded shape: (8418, 211)
appended-encoded shape: (8418, 581)

Train shape: (4209, 580)
Test shape: (4209, 579)


In [3]:
# Calculate additional features: dimensionality reduction components
n_comp=10 # was 10

# uncomment to scale data before applying decompositions
# however, all features are binary (in [0,1] interval), i don't know if it's worth something
train_scaled = train.drop('y', axis=1).copy()
test_scaled = test.copy()
'''
ss = StandardScaler()
ss.fit(train.drop('y', axis=1))

train_scaled = ss.transform(train.drop('y', axis=1).copy())
test_scaled = ss.transform(test.copy())
'''

# PCA
pca = PCA(n_components=n_comp, random_state=seed)
pca2_results_train = pca.fit_transform(train_scaled)
pca2_results_test = pca.transform(test_scaled)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train_scaled)
ica2_results_test = ica.transform(test_scaled)

# Append it to dataframes
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-1]
    test['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    train['ica_' + str(i)] = ica2_results_train[:,i-1]
    test['ica_' + str(i)] = ica2_results_test[:, i-1]

/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [4]:
# create augmentation by feature importances as additional features
t = train['y']
tr = train.drop(['y'], axis=1)

# Tree-based estimators can be used to compute feature importances
clf = GradientBoostingRegressor(
                max_depth=4, 
                learning_rate=0.005, 
                random_state=seed, 
                subsample=0.95, 
                n_estimators=200
)

# fit regressor
clf.fit(tr, t)

# df to hold feature importances
features = pd.DataFrame()
features['feature'] = tr.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

# select best features
model = SelectFromModel(clf, prefit=True)
train_reduced = model.transform(tr)


test_reduced = model.transform(test.copy())

# dataset augmentation
train = pd.concat([train, pd.DataFrame(train_reduced)], axis=1)
test = pd.concat([test, pd.DataFrame(test_reduced)], axis=1)

# check new shape
print('\nTrain shape: {}\nTest shape: {}'.format(train.shape, test.shape))


Train shape: (4209, 616)
Test shape: (4209, 615)


In [5]:
# X, y preparation
X, y = train.drop('y', axis=1).values, train.y.values
print(X.shape)

# X_test preparation
X_test = test
print(X_test.shape)

# X_test preparation
X_test = test
print(X_test.shape)

# train/validation split
X_tr, X_val, y_tr, y_val = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=seed
)

(4209, 615)
(4209, 615)
(4209, 615)


In [6]:
# define custom R2 metrics for Keras backend
from keras import backend as K

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# base model architecture definition
def model(dropout_rate=0.0, act_func='relu'):
    model = Sequential()
    #input layer
    model.add(Dense(input_dims, input_dim=input_dims))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout_rate))
    # hidden layers
    model.add(Dense(input_dims))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(dropout_rate))
         
    model.add(Dense(input_dims//2))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(input_dims//4, activation=act_func))
    
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  optimizer='adam',
                  metrics=[r2_keras] # you can add several if needed
                 )
    
    # Visualize NN architecture
    #print(model.summary())
    return model

In [15]:
# initialize input dimension
input_dims = train.shape[1]-1


# make np.seed fixed
np.random.seed(seed)

# initialize estimator, wrap model in KerasRegressor
keras_estimator = KerasRegressor(
    build_fn=model, 
    verbose=1
)

# define path to save model
import os
model_path = 'keras_model.h5'

# define the grid search parameters
#batch_size = [20, 10, 15, 25]          
#epochs = [35]
#act_func = ['softmax', 'linear']
#dropout_rate = [0.6, 0.8, 0.7, 0.4]

batch_size = [20]          
epochs = [2]
act_func = ['linear']
dropout_rate = [0.6]
param_grid = dict(batch_size=batch_size, epochs=epochs, dropout_rate=dropout_rate, act_func = act_func)

In [16]:
grid = GridSearchCV(estimator=keras_estimator, 
                    param_grid=param_grid, cv = 5, verbose = 1)
grid_result = grid.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Epoch 1/2
3367/3367 [==============================] - 2s - loss: 7166.4629 - r2_keras: -56.1403     
Epoch 2/2
3367/3367 [==============================] - 2s - loss: 6990.6180 - r2_keras: -56.4005     
Epoch 2/2
3367/3367 [==============================] - 2s - loss: 7017.1991 - r2_keras: -54.6932     
Epoch 2/2
3367/3367 [==============================] - 2s - loss: 6938.2279 - r2_keras: -52.6095     
Epoch 2/2
3368/3368 [==============================] - 2s - loss: 7218.6733 - r2_keras: -54.8250     
Epoch 2/2
3280/3368 [============================>.] - ETA: 0s

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   29.9s finished


Epoch 1/2
4209/4209 [==============================] - 3s - loss: 6049.7202 - r2_keras: -46.7737     
Epoch 2/2
4209/4209 [==============================] - 2s - loss: 945.5262 - r2_keras: -6.5737     
Best: 196.483394 using {'epochs': 2, 'dropout_rate': 0.6, 'act_func': 'linear', 'batch_size': 20}
196.483394 (46.419301) with: {'epochs': 2, 'dropout_rate': 0.6, 'act_func': 'linear', 'batch_size': 20}


In [7]:


grid = GridSearchCV(estimator=keras_estimator, 
                    param_grid=param_grid, cv = 5
                    #, n_jobs=-1
                    , verbose = 1)
grid_result = grid.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#resource exhausted: oom when
#happen after 8 hours

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Epoch 1/35
3367/3367 [==============================] - 7s - loss: 10237.0972 - r2_keras: -80.6584     
Epoch 2/35
3367/3367 [==============================] - 2s - loss: 10161.5386 - r2_keras: -77.0226     
Epoch 3/35
3367/3367 [==============================] - 2s - loss: 10083.5896 - r2_keras: -78.9569     
Epoch 4/35
3367/3367 [==============================] - 2s - loss: 10002.8144 - r2_keras: -77.1584     
Epoch 5/35
3367/3367 [==============================] - 2s - loss: 9920.1685 - r2_keras: -76.6803     
Epoch 6/35
3367/3367 [==============================] - 2s - loss: 9835.7949 - r2_keras: -75.2444     
Epoch 7/35
3367/3367 [==============================] - 2s - loss: 9748.1251 - r2_keras: -75.2693     
Epoch 8/35
3367/3367 [==============================] - 2s - loss: 9657.0784 - r2_keras: -74.5438     
Epoch 9/35
3367/3367 [==============================] - 2s - loss: 9565.1971 - r2_keras: -72.0500     
Epoch 1

3368/3368 [==============================] - 2s - loss: 9082.0400 - r2_keras: -67.9189     
Epoch 16/35
3368/3368 [==============================] - 2s - loss: 8994.2305 - r2_keras: -67.6312     
Epoch 17/35
3368/3368 [==============================] - 2s - loss: 8905.6019 - r2_keras: -66.6918     
Epoch 18/35
3368/3368 [==============================] - 2s - loss: 8816.8123 - r2_keras: -65.9725     
Epoch 19/35
3368/3368 [==============================] - 2s - loss: 8733.5002 - r2_keras: -65.4402     
Epoch 20/35
3368/3368 [==============================] - 2s - loss: 8655.7284 - r2_keras: -65.8670     
Epoch 21/35
3368/3368 [==============================] - 2s - loss: 8582.0709 - r2_keras: -62.7637     
Epoch 22/35
3368/3368 [==============================] - 2s - loss: 8511.6516 - r2_keras: -64.3407     
Epoch 23/35
3368/3368 [==============================] - 2s - loss: 8443.5024 - r2_keras: -64.7024     
Epoch 24/35
3368/3368 [==============================] - 2s - loss: 8377.163

3367/3367 [==============================] - 2s - loss: 7976.8582 - r2_keras: -59.7205     
Epoch 30/35
3367/3367 [==============================] - 2s - loss: 7916.7960 - r2_keras: -61.0473     
Epoch 31/35
3367/3367 [==============================] - 2s - loss: 7857.1853 - r2_keras: -60.9259     
Epoch 32/35
3367/3367 [==============================] - 2s - loss: 7798.0789 - r2_keras: -60.9081     
Epoch 33/35
3367/3367 [==============================] - 2s - loss: 7739.4132 - r2_keras: -59.5219     
Epoch 34/35
3367/3367 [==============================] - 2s - loss: 7681.1138 - r2_keras: -61.1342     
Epoch 35/35
3368/3368 [==============================] - 2s - loss: 10346.1618 - r2_keras: -78.0644     
Epoch 2/35
3368/3368 [==============================] - 2s - loss: 10269.9710 - r2_keras: -78.7734     
Epoch 3/35
3368/3368 [==============================] - 2s - loss: 10191.7311 - r2_keras: -78.0508     
Epoch 4/35
3368/3368 [==============================] - 2s - loss: 10111.25

3367/3367 [==============================] - 2s - loss: 9674.1577 - r2_keras: -74.5346     
Epoch 9/35
3367/3367 [==============================] - 2s - loss: 9581.9021 - r2_keras: -74.4507     
Epoch 10/35
3367/3367 [==============================] - 2s - loss: 9483.7222 - r2_keras: -72.7822     
Epoch 11/35
3367/3367 [==============================] - 2s - loss: 9381.4533 - r2_keras: -72.2771     
Epoch 12/35
3367/3367 [==============================] - 2s - loss: 9275.8707 - r2_keras: -71.7209     
Epoch 13/35
3367/3367 [==============================] - 2s - loss: 9170.3673 - r2_keras: -70.3822     
Epoch 14/35
3367/3367 [==============================] - 2s - loss: 9070.4112 - r2_keras: -70.2327     
Epoch 15/35
3367/3367 [==============================] - 2s - loss: 8974.6840 - r2_keras: -69.0775     
Epoch 16/35
3367/3367 [==============================] - 2s - loss: 8880.1891 - r2_keras: -68.8445     
Epoch 17/35
3367/3367 [==============================] - 2s - loss: 8791.9531

3367/3367 [==============================] - 2s - loss: 8378.1205 - r2_keras: -64.9794     
Epoch 24/35
3367/3367 [==============================] - 2s - loss: 8314.3553 - r2_keras: -64.4178     
Epoch 25/35
3367/3367 [==============================] - 2s - loss: 8251.3365 - r2_keras: -63.0770     
Epoch 26/35
3367/3367 [==============================] - 2s - loss: 8189.2608 - r2_keras: -66.2267     
Epoch 27/35
3367/3367 [==============================] - 2s - loss: 8127.8401 - r2_keras: -62.1284     
Epoch 28/35
3367/3367 [==============================] - 2s - loss: 8067.0919 - r2_keras: -61.8019     
Epoch 29/35
3367/3367 [==============================] - 2s - loss: 8006.8024 - r2_keras: -62.3765     
Epoch 30/35
3367/3367 [==============================] - 2s - loss: 7947.0839 - r2_keras: -60.6158     
Epoch 31/35
3367/3367 [==============================] - 2s - loss: 7887.7110 - r2_keras: -59.7368     
Epoch 32/35
3367/3367 [==============================] - 2s - loss: 7828.786

...

3367/3367 [==============================] - 2s - loss: 517.6736 - r2_keras: -2.8062     
Epoch 11/35
3367/3367 [==============================] - 2s - loss: 469.8988 - r2_keras: -2.3854     
Epoch 12/35
3367/3367 [==============================] - 2s - loss: 442.3843 - r2_keras: -2.2891     
Epoch 13/35
3367/3367 [==============================] - 2s - loss: 419.0792 - r2_keras: -2.0584     
Epoch 14/35
3367/3367 [==============================] - 2s - loss: 394.5648 - r2_keras: -1.7960     
Epoch 15/35
3367/3367 [==============================] - 2s - loss: 363.8162 - r2_keras: -1.6641     
Epoch 16/35
3367/3367 [==============================] - 2s - loss: 353.6245 - r2_keras: -1.6090     
Epoch 17/35
3367/3367 [==============================] - 2s - loss: 336.2558 - r2_keras: -1.4323     
Epoch 18/35
3367/3367 [==============================] - 2s - loss: 335.6497 - r2_keras: -1.3393     
Epoch 19/35
3367/3367 [==============================] - 2s - loss: 308.8532 - r2_keras: -1.18

3367/3367 [==============================] - 2s - loss: 171.6750 - r2_keras: -0.2755     
Epoch 29/35
3367/3367 [==============================] - 2s - loss: 159.0478 - r2_keras: -0.1637     
Epoch 30/35
3367/3367 [==============================] - 2s - loss: 156.5602 - r2_keras: -0.1707     
Epoch 31/35
3367/3367 [==============================] - 2s - loss: 160.5559 - r2_keras: -0.1865     
Epoch 32/35
3367/3367 [==============================] - 2s - loss: 143.9333 - r2_keras: -0.0614     
Epoch 33/35
3367/3367 [==============================] - 2s - loss: 145.7348 - r2_keras: -0.0676     
Epoch 34/35
3367/3367 [==============================] - 2s - loss: 141.5978 - r2_keras: -0.0370     
Epoch 35/35
3367/3367 [==============================] - 10s - loss: 8424.7626 - r2_keras: -63.7572    
Epoch 2/35
3367/3367 [==============================] - 2s - loss: 1995.3477 - r2_keras: -14.0686     
Epoch 3/35
3367/3367 [==============================] - 2s - loss: 1165.1352 - r2_keras: -7

3367/3367 [==============================] - 2s - loss: 211.8196 - r2_keras: -0.6242     
Epoch 11/35
3367/3367 [==============================] - 2s - loss: 210.3616 - r2_keras: -0.5489     
Epoch 12/35
3367/3367 [==============================] - 2s - loss: 201.1403 - r2_keras: -0.4839     
Epoch 13/35
3367/3367 [==============================] - 2s - loss: 182.9720 - r2_keras: -0.3793     
Epoch 14/35
3367/3367 [==============================] - 2s - loss: 169.7419 - r2_keras: -0.2774     
Epoch 15/35
3367/3367 [==============================] - 2s - loss: 167.6825 - r2_keras: -0.2487     
Epoch 16/35
3367/3367 [==============================] - 2s - loss: 166.4367 - r2_keras: -0.2739     
Epoch 17/35
3367/3367 [==============================] - 2s - loss: 153.9591 - r2_keras: -0.1443     
Epoch 18/35
3367/3367 [==============================] - 2s - loss: 149.1625 - r2_keras: -0.1092     
Epoch 19/35
3367/3367 [==============================] - 2s - loss: 141.6707 - r2_keras: -0.05

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed: 310.5min finished


Epoch 1/35
4209/4209 [==============================] - 11s - loss: 10255.3647 - r2_keras: -78.4747    
Epoch 2/35
4209/4209 [==============================] - 2s - loss: 10179.9677 - r2_keras: -74.7071     
Epoch 3/35
4209/4209 [==============================] - 2s - loss: 10102.3007 - r2_keras: -74.6852     
Epoch 4/35
4209/4209 [==============================] - 2s - loss: 10021.4896 - r2_keras: -73.7197     
Epoch 5/35
4209/4209 [==============================] - 2s - loss: 9938.2542 - r2_keras: -73.6973     
Epoch 6/35
4209/4209 [==============================] - 2s - loss: 9853.4283 - r2_keras: -75.4425     
Epoch 7/35
4209/4209 [==============================] - 3s - loss: 9766.9947 - r2_keras: -71.8538     
Epoch 8/35
4209/4209 [==============================] - 3s - loss: 9678.0685 - r2_keras: -70.4360     
Epoch 9/35
4209/4209 [==============================] - 2s - loss: 9585.3923 - r2_keras: -71.2014     
Epoch 10/35
4209/4209 [==============================] - 3s - loss: 9

In [8]:
grid_result.cv_results_

{'mean_fit_time': array([  81.83121686,   81.00144877,   80.92504983,   82.44831653,
         157.99503055,  159.40115347,  160.39647603,  160.67076998,
         111.74938474,  112.65685005,  114.21254029,  115.30545287,
          74.3210288 ,   74.42918205,   75.52205787,   76.79394984,
          85.98990035,   87.62378464,   88.97916117,   90.34156256,
         169.0146338 ,  167.95908213,  170.61066375,  171.98583193,
         121.26625471,  122.52279758,  123.92474871,  126.06736398,
          81.19130898,   81.84158115,   83.22685246,   82.72435513]),
 'mean_score_time': array([ 0.29308929,  0.50064292,  0.71261888,  0.93351808,  1.28309193,
         1.50147681,  1.7337389 ,  1.93217998,  2.08367977,  2.31191964,
         2.51865702,  2.7614748 ,  2.91229243,  3.13712807,  3.3675683 ,
         3.57280574,  3.81631575,  4.03663363,  4.2296824 ,  4.44975572,
         4.86446009,  5.08055439,  5.3510345 ,  5.56846614,  5.70170393,
         6.00883641,  6.27808809,  6.53595791,  6.671

In [ ]:
# if best iteration's model was saved then load and use it
if os.path.isfile(model_path):
    estimator = load_model(model_path, custom_objects={'r2_keras': r2_keras})

# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y_tr, estimator.predict(X_tr))**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y_tr, estimator.predict(X_tr)))) # R^2 train

# check performance on validation set
print('MSE val: {}'.format(mean_squared_error(y_val, estimator.predict(X_val))**0.5)) # mse val
print('R^2 val: {}'.format(r2_score(y_val, estimator.predict(X_val)))) # R^2 val
pass

# predict results
res = estimator.predict(X_test.values).ravel()

# create df and convert it to csv
output = pd.DataFrame({'id': id_test, 'y': res})
output.to_csv('keras-baseline.csv', index=False)